# Srq2Seq Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [ ]:
class Seq2Seq:
    def __init__(self, input_vocab_size, target_vocab_size, input_length, target_length, hidden_units):
        self.input_vocab_size = input_vocab_size
        self.target_vocab_size = target_vocab_size
        self.input_length = input_length
        self.target_length = target_length
        self.hidden_units = hidden_units
        
        # Define encoder
        encoder_inputs = Input(shape=(self.input_length,))
        encoder_embedding = tf.keras.layers.Embedding(self.input_vocab_size, self.hidden_units, input_length=self.input_length)(encoder_inputs)
        encoder_lstm = LSTM(self.hidden_units, return_state=True)
        encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
        encoder_states = [state_h, state_c]

        # Define decoder
        decoder_inputs = Input(shape=(self.target_length,))
        decoder_embedding = tf.keras.layers.Embedding(self.target_vocab_size, self.hidden_units, input_length=self.target_length)(decoder_inputs)
        decoder_lstm = LSTM(self.hidden_units, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
        decoder_dense = Dense(self.target_vocab_size, activation='softmax')
        decoder_outputs = decoder_dense(decoder_outputs)

        # Define model
        self.model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
        self.model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    def train(self, input_sequences, target_sequences, batch_size, epochs):
        self.model.fit([input_sequences, target_sequences[:, :-1]], target_sequences[:, 1:], batch_size=batch_size, epochs=epochs, validation_split=0.2)

    def predict(self, input_sequence):
        return self.model.predict(input_sequence)